In [ ]:
%load_ext autoreload
%autoreload 2
    
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from glob import glob
from ConvFitterVDWrapper import ConvFitterVDWrapper
from waffles.data_classes.EasyWaveformCreator import EasyWaveformCreator
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.np02_utils.AutoMap import dict_endpoints_channels_list, dict_module_to_uniqch, dict_uniqch_to_module, strUch
from waffles.np02_utils.AutoMap import ordered_channels_cathode, ordered_channels_membrane, ordered_modules_cathode, ordered_modules_membrane
from waffles.np02_utils.PlotUtils import matplotlib_plot_WaveformSetGrid
from waffles.np02_utils.load_utils import ch_read_template

from utils_plot import plot_fit
from utils_conv import process_convfit
from larfitter import retrieve_responses

import mplhep
mplhep.style.use(mplhep.style.ROOT)
plt.rcParams.update({'font.size': 20,
                     'grid.linestyle': '--',
                     'axes.grid': True,
                     'figure.autolayout': True,
                     'figure.figsize': [14,6]
                     })


In [ ]:
run = 39510 

response_root_path = "/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/xenon_averages/average_waveforms-hvieirad/"


response_folder_cathode = Path(response_root_path + f'run{run:06d}_cathode_response')
response_folder_membrane = Path(response_root_path + f'run{run:06d}_membrane_response')

In [ ]:
templates = ch_read_template('templates_large_pulses')

In [ ]:
responses = {}
counts = {}
retrieve_responses(response_folder_cathode, responses, counts)
retrieve_responses(response_folder_membrane, responses, counts)

In [ ]:
cfitparams = dict(
    threshold_align_template=0.27,
    threshold_align_response=0.2,
    error=10,
    dointerpolation=True,
    interpolation_factor=8,
    align_waveforms = True,
    dtime=16,
    convtype='fft',
    usemplhep=True,
    scinttype = "lar"
)
cfit = {}
dict_ep_ch_with_both_signals = {}
for ep, ch_wvf in responses.items():
    for ch, _ in ch_wvf.items():
        if ep in templates.keys():
            cfit[ep] = cfit.get(ep, {})
            if ch in templates[ep].keys():
                dict_ep_ch_with_both_signals[ep] = dict_ep_ch_with_both_signals.get(ep, []) + [ch]
                cfit[ep][ch] = ConvFitterVDWrapper(**cfitparams)

In [ ]:
dict_ok_membrane = { 107: dict_ep_ch_with_both_signals[107] if 107 in dict_ep_ch_with_both_signals else [] }
dict_ok_cathode = { 106: dict_ep_ch_with_both_signals[106] if 106 in dict_ep_ch_with_both_signals else []}

In [ ]:
wfset = EasyWaveformCreator.create_WaveformSet_dictEndpointCh(dict_endpoint_ch=dict_endpoints_channels_list, run_number=run)


In [ ]:
module_test = "C2(1)" # ex: C2(2)

if module_test:
    ep = dict_module_to_uniqch[module_test].endpoint
    ch = dict_module_to_uniqch[module_test].channel
else:
    ep = 106
    ch = 30


tmpcfit = cfit[ep][ch]
tmpcfit.set_template_waveform(templates[ep][ch])
tmpcfit.set_response_waveform(responses[ep][ch])
plt.plot(tmpcfit.template, label='template')
plt.plot(tmpcfit.response*np.max(tmpcfit.template)/np.max(tmpcfit.response), label='normalized-response')
plt.title(f"{dict_uniqch_to_module[strUch(ep,ch)]}: {ep}-{ch}")
plt.legend()
tmpcfit.prepare_waveforms()
plt.figure()
plt.plot(tmpcfit.template, label='template')
plt.plot(tmpcfit.response*np.max(tmpcfit.template)/np.max(tmpcfit.response), label='normalized-response')
plt.legend()
# plt.xlim(300,600)

In [ ]:
tmpcfit.convtype = "fft"
process_convfit(
    ep, ch,
    response=responses[ep][ch],
    template=templates[ep][ch],
    cfitch=tmpcfit,
    print_flag=True,
    scan=8
)

tmpcfit.plot()
# plt.xlim(500,1500)
plt.legend()

In [ ]:

fig, ax = tmpcfit.m.draw_mnmatrix(figsize=[16,6], cl=[0.68,0.954])

In [ ]:
paramsplot = {
    'cfit':cfit,
    'dofit':True,
    'responses': responses,
    'templates': templates,
    'verbose': False,
    'scan': 8
}

fig, axs = matplotlib_plot_WaveformSetGrid(wfset, detector=ordered_modules_cathode, plot_function=plot_fit, func_params=paramsplot, cols=4, figsize=(32,32))
Path('./graphs').mkdir(exist_ok=True)
plt.savefig(f"./graphs/convfit_results_cathode_run{run:06d}")
plt.show()

In [ ]:
fig, axs = matplotlib_plot_WaveformSetGrid(wfset, detector=ordered_modules_membrane, plot_function=plot_convfit, func_params=paramsplot, cols=4, figsize=(32,32))
plt.savefig(f"./graphs/fit_results_membrane_run{run:06d}")
plt.show()